In [1]:
import os, re, json
import pandas as pd
from textwrap import dedent
from dotenv import load_dotenv
from openai import OpenAI
import base64, requests, unicodedata
from io import BytesIO
from PIL import Image
from pathlib import Path

load_dotenv()  # toma OPENAI_API_KEY del .env si existe


True

In [2]:
from pathlib import Path

# 👇 Ajustá la ruta si hace falta
CSV_PATH = "../data/base_conocimiento_afiliaciones_clean.csv"

p = Path(CSV_PATH)
if not p.exists():
    raise FileNotFoundError(f"CSV no encontrado: {p.resolve()} — ajustá CSV_PATH.")

df = pd.read_csv(p, dtype=str, keep_default_na=False)
df = df[df["estado"].str.lower().isin(["vigente","en revisión"])].copy()

for col in ["id","titulo","contenido","respuesta_validada","palabras_clave"]:
    if col in df.columns:
        df[col] = df[col].fillna("").astype(str)

print("Filas cargadas:", len(df))


Filas cargadas: 18


In [3]:
def dividir_en_tokens(texto: str):
    texto = (texto or "").lower()
    return [t for t in re.split(r"[^a-záéíóúñü0-9]+", texto) if t]

def calcular_relevancia(tokens_consulta, fila):
    puntaje = 0
    puntaje += 3 * len(set(tokens_consulta) & set(dividir_en_tokens(fila.get("palabras_clave",""))))
    puntaje += 2 * len(set(tokens_consulta) & set(dividir_en_tokens(fila.get("titulo",""))))
    puntaje += 1 * len(set(tokens_consulta) & set(dividir_en_tokens(fila.get("contenido",""))))
    return puntaje

def buscar_faq(consulta: str, tabla: pd.DataFrame):
    toks = dividir_en_tokens(consulta)
    puntuadas = [(calcular_relevancia(toks, fila), idx) for idx, fila in tabla.iterrows()]
    puntuadas = [(p,i) for p,i in puntuadas if p>0]
    if not puntuadas:
        return None
    puntuadas.sort(reverse=True)
    return tabla.loc[puntuadas[0][1]]


In [4]:
def compactar_texto(s: str, max_chars=800):
    s = re.sub(r"\s+", " ", (s or "")).strip()
    return s[:max_chars]

# a) System prompt fijo (v6)
SYSTEM_PROMPT_V6 = dedent("""
Rol: Asistente de Afiliaciones de IOMA. Público: agentes. Tono: institucional.

Instrucciones:
- "checklist": SOLO de <<BASE>>. Si falta dato: "No consta en la normativa adjunta".
- "terminos_clave" y "objetivo_y_buenas_practicas": podés usar conocimiento externo.
- Responder SOLO con JSON válido.

Formato:
{
  "checklist": ["...", "..."],
  "terminos_clave": ["Término: definición breve", "...", "..."],
  "objetivo_y_buenas_practicas": ["Buena práctica: detalle breve", "...", "..."],
  "cierre": "Fuente: base de conocimiento vigente"
}
""").strip()

# b) Contexto dinámico por turno (Pregunta + BASE)
def build_context_v6(fila, pregunta: str):
    if fila is None:
        base = ""
        idtitulo = "(ninguna)"
    else:
        base = (fila.get("respuesta_validada") or 
                fila.get("contenido") or 
                fila.get("titulo","")).strip()
        base = compactar_texto(base, 800)
        idtitulo = f"{fila.get('id','(sin id)')} – {fila.get('titulo','(sin título)')}"
    payload = dedent(f'''
    Pregunta: "{pregunta}"

    <<BASE>>
    {base}
    <<FIN_BASE>>
    ''').strip()
    return payload, idtitulo


In [5]:
api_key = os.getenv("OPENAI_API_KEY")
assert api_key, "Falta OPENAI_API_KEY (definila en .env o variable de entorno)."

client = OpenAI(api_key=api_key)
MODEL = "gpt-4o"  # o "gpt-4o"
print("Cliente listo, modelo:", MODEL)


Cliente listo, modelo: gpt-4o


In [6]:
# Paleta IOMA
PALETA_IOMA = {
    "teal":    "#2D8DA6",
    "purpura": "#6A5AAE",
    "magenta": "#C4286F",
    "blanco":  "#FFFFFF",
    "gris":    "#3C3C3C",
}

IMGS_DIR = Path("imgs")
IMGS_DIR.mkdir(parents=True, exist_ok=True)

def slugify(s: str) -> str:
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s = re.sub(r"[^a-zA-Z0-9]+", "-", s).strip("-").lower()
    return s or "imagen"

def resize_to(path_in: Path, path_out: Path, size=(256, 256)):
    im = Image.open(path_in).convert("RGB")
    im = im.resize(size, Image.LANCZOS)
    im.save(path_out, format="PNG")
    return path_out


In [7]:
IMAGES_MODEL_PRIMARY   = "dall-e-3"
IMAGES_MODEL_FALLBACK  = "gpt-image-1"

def build_prompt_imagen(tema: str) -> str:
    return f"""
Generar una ilustración institucional clara y elegante sobre **{tema}** como figura/escena central,
con un fondo armónico en degradado de la paleta IOMA (teal {PALETA_IOMA['teal']}, púrpura {PALETA_IOMA['purpura']}, magenta {PALETA_IOMA['magenta']}).
Incluir un halo luminoso o un marco circular sutil detrás del elemento principal para reforzar el foco.
Estilo minimalista, moderno y cálido, sin texto en la imagen. Composición limpia.
"""

def _generate_image_bytes(model: str, prompt_text: str, size: str = "1024x1024"):
    resp = client.images.generate(model=model, prompt=prompt_text, n=1, size=size)
    # Intentar URL
    try:
        url = resp.data[0].url
        if url:
            return requests.get(url, timeout=120).content
    except Exception:
        pass
    # Intentar b64_json
    try:
        b64 = resp.data[0].b64_json
        if b64:
            return base64.b64decode(b64)
    except Exception:
        pass
    raise RuntimeError("No se pudo obtener la imagen (ni url ni b64_json).")

def get_or_create_image_for_theme(tema: str,
                                  size_generate="1024x1024",
                                  size_display=(256, 256)) -> Path:
    slug = slugify(tema)
    p_full = IMGS_DIR / f"{slug}_1024.png"
    p_disp = IMGS_DIR / f"{slug}_{size_display[0]}x{size_display[1]}.png"

    if p_disp.exists():
        return p_disp
    if p_full.exists():
        return resize_to(p_full, p_disp, size=size_display)

    prompt_text = build_prompt_imagen(tema)
    try:
        img_bytes = _generate_image_bytes(IMAGES_MODEL_PRIMARY, prompt_text, size_generate)
    except Exception:
        img_bytes = _generate_image_bytes(IMAGES_MODEL_FALLBACK, prompt_text, size_generate)

    Image.open(BytesIO(img_bytes)).convert("RGB").save(p_full)
    resize_to(p_full, p_disp, size=size_display)
    return p_disp


In [8]:
def infer_tema_imagen(consulta: str, fila_sel) -> str:
    # override manual si el usuario escribe 'tema: ...'
    m = re.search(r"tema\s*:\s*([^\n\r]+)", consulta, flags=re.IGNORECASE)
    if m:
        override = m.group(1).strip()
        if override:
            return override

    cand = (fila_sel.get("titulo","") if isinstance(fila_sel, dict) else (fila_sel["titulo"] if fila_sel is not None and "titulo" in fila_sel else "")) or ""
    texto_ref = f"{consulta} {cand}".lower()

    rules = [
        (r"reci[eé]n\s*nacid[oa]", "afiliación de recién nacido/a"),
        (r"recien\s*nac", "afiliación de recién nacido/a"),
        (r"estudiante", "afiliación de estudiante"),
        (r"conviviente", "afiliación de conviviente"),
        (r"c[oó]nyuge|conyuge", "afiliación de cónyuge"),
        (r"monotribut", "afiliación de monotributista"),
        (r"padre|madre|progenitor", "afiliación por vínculo familiar"),
    ]
    for pat, tema in rules:
        if re.search(pat, texto_ref):
            return tema

    return cand.strip() or "afiliaciones IOMA"


In [9]:
# Historial con v6
chat_history = [{"role": "system", "content": SYSTEM_PROMPT_V6}]
last_meta = {"fila_sel": None, "faq_idtitulo": None, "tema": None}

def reset_history():
    global chat_history, last_meta
    chat_history = [{"role": "system", "content": SYSTEM_PROMPT_V6}]
    last_meta = {"fila_sel": None, "faq_idtitulo": None, "tema": None}
    return "Historial reiniciado (v6 cargada)."

def chat(consulta: str, temperature: float = 0.2, max_tokens: int = 400):
    global last_meta

    fila_sel = buscar_faq(consulta, df)
    contexto, idtitulo = build_context_v6(fila_sel, consulta)

    chat_history.append({"role": "user", "content": contexto})
    resp = client.chat.completions.create(
        model=MODEL,
        messages=chat_history,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    assistant_msg = resp.choices[0].message.content
    chat_history.append({"role": "assistant", "content": assistant_msg})

    tema = infer_tema_imagen(consulta, fila_sel.to_dict() if fila_sel is not None else {})
    last_meta = {
        "fila_sel": (fila_sel.to_dict() if fila_sel is not None else None),
        "faq_idtitulo": idtitulo,
        "tema": tema
    }
    return assistant_msg


In [10]:
# UI con estilos IOMA + banner + respuesta coloreada + imagen centrada
try:
    import ipywidgets as widgets
    from IPython.display import display
    from pathlib import Path
    import html

    banner = widgets.HTML(
        value=f"""
        <div style="
            padding:14px 16px;
            border-radius:12px;
            background: linear-gradient(90deg, {PALETA_IOMA['teal']}22, {PALETA_IOMA['purpura']}22, {PALETA_IOMA['magenta']}22);
            border:1px solid {PALETA_IOMA['teal']};
            ">
          <div style="font-weight:700; font-size:16px; color:{PALETA_IOMA['teal']};">
            Bienvenido/a al Asistente de Afiliaciones
          </div>
          <div style="font-size:12.5px; color:{PALETA_IOMA['gris']};">
            Consultá requisitos, documentación y buenas prácticas. Las respuestas siguen la BASE vigente y se devuelven en JSON.
          </div>
        </div>
        """
    )

    input_box = widgets.Textarea(
        value='',
        placeholder='Ej: recién nacido / estudiante / conviviente (podés usar "tema: estudiante" para forzar)',
        description='Usuario:',
        disabled=False,
        layout=widgets.Layout(width='100%', height='80px')
    )
    send_btn = widgets.Button(description='Enviar')
    reset_btn = widgets.Button(description='Reset historial')

    out = widgets.Output(layout=widgets.Layout(
        border=f'2px solid {PALETA_IOMA["teal"]}',
        padding='8px',
        max_height='520px',
        overflow='auto'
    ))

    try:
        send_btn.style.button_color  = PALETA_IOMA["teal"]
        reset_btn.style.button_color = PALETA_IOMA["magenta"]
    except Exception:
        send_btn.button_style  = 'info'
        reset_btn.button_style = 'warning'

    def on_send_clicked(_):
        with out:
            user_text = input_box.value.strip()
            if not user_text:
                print("Escribí un mensaje primero.")
                return

            display(widgets.HTML(
                f"<div style='margin-top:10px; font-weight:600; color:{PALETA_IOMA['purpura']};'>Usuario: {html.escape(user_text)}</div>"
            ))

            ans = chat(user_text)

            faq_label = html.escape(last_meta.get('faq_idtitulo','(ninguna)') or '(ninguna)')
            ans_html = html.escape(ans)
            left_panel = widgets.HTML(
                value=f"""
                <div style="font-family: Segoe UI, Roboto, Arial, sans-serif; color:{PALETA_IOMA['gris']}; line-height:1.45;">
                  <div style="font-size:12px; opacity:.8;">FAQ seleccionada: {faq_label}</div>
                  <div style="font-weight:700; margin:6px 0 8px 0; color:{PALETA_IOMA['teal']};">Asistente</div>
                  <pre style="
                      white-space:pre-wrap;
                      background:#FFFFFF;
                      color:{PALETA_IOMA['gris']};
                      border:1px solid {PALETA_IOMA['teal']};
                      border-radius:10px;
                      padding:12px;
                      box-shadow: 0 1px 4px rgba(0,0,0,.06);
                      ">{ans_html}</pre>
                </div>
                """
            )

            try:
                tema = last_meta.get("tema") or "afiliaciones IOMA"
                path_256 = get_or_create_image_for_theme(
                    tema, size_generate="1024x1024", size_display=(256, 256)
                )
                img_bytes = Path(path_256).read_bytes()
                img_widget = widgets.Image(value=img_bytes, format='png', width=256, height=256)
                caption = widgets.HTML(
                    f"<div style='text-align:center;color:{PALETA_IOMA['gris']};font-size:12px;'>Imagen: {html.escape(tema)}</div>"
                )
                right_panel = widgets.VBox(
                    [img_widget, caption],
                    layout=widgets.Layout(
                        align_items='center',
                        width='30%',
                        align_self='center'   # centra verticalmente
                    )
                )
            except Exception as e:
                right_panel = widgets.HTML(
                    f"<div style='color:{PALETA_IOMA['magenta']};'>No se pudo mostrar la imagen ({html.escape(str(e))}).</div>"
                )

            display(
                widgets.HBox(
                    [
                        widgets.VBox([left_panel],  layout=widgets.Layout(width='70%')),
                        right_panel
                    ],
                    layout=widgets.Layout(width='100%', align_items='center')
                )
            )

            input_box.value = ''

    def on_reset_clicked(_):
        with out:
            print(reset_history())

    send_btn.on_click(on_send_clicked)
    reset_btn.on_click(on_reset_clicked)

    display(banner, input_box, widgets.HBox([send_btn, reset_btn]), out)

except Exception as e:
    print("UI opcional no disponible:", e)


HTML(value='\n        <div style="\n            padding:14px 16px;\n            border-radius:12px;\n         …

Textarea(value='', description='Usuario:', layout=Layout(height='80px', width='100%'), placeholder='Ej: recién…

Output(layout=Layout(border_bottom='2px solid #2D8DA6', border_left='2px solid #2D8DA6', border_right='2px sol…